In [ ]:
import os
import sys
from pathlib import Path
from matplotlib.pyplot import imshow
import numpy as np
import torch
from scipy.ndimage import zoom
from scipy import ndimage
from model.MGUnet import MGUNet
from util import read_yml
from util.reader import reader
from scipy import ndimage
from albumentations import functional


cfg_path, img_folder, ckpath, out_dir, input_size = ["/yeping/new/AL-ACDC/EXP/ISIC_exp/SSL/0.03/Baseline/config.yml", "/yeping/new/AL-ACDC/data/ISIC/val/images/" ,"/yeping/new/AL-ACDC/EXP/ISIC_exp/SSL/0.03/Baseline/checkpoint/c0_best0.8198.pt", "/yeping/new/AL-ACDC/EXP/ISIC_exp/SSL/0.03/Baseline/inference/" ,"416"]
input_size = int(input_size)
assert os.path.exists(img_folder) and os.path.exists(ckpath)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

os.makedirs(out_dir, exist_ok=True)

cfg_path = read_yml(cfg_path)
model = MGUNet(cfg_path["Network"]).to(device)

model.load_state_dict(torch.load(ckpath, map_location=device)["model_state_dict"])
model.eval()

In [ ]:
def get_largest_k_components(image, k=1):
    """
    Get the largest K components from 2D or 3D binary image.

    :param image: The input ND array for binary segmentation.
    :param k: (int) The value of k.

    :return: An output array with only the largest K components of the input.
    """
    dim = len(image.shape)
    if (image.sum() == 0):
        print('the largest component is null')
        return image
    if (dim < 2 or dim > 3):
        raise ValueError("the dimension number should be 2 or 3")
    s = ndimage.generate_binary_structure(dim, 1)
    labeled_array, numpatches = ndimage.label(image, s)
    sizes = ndimage.sum(image, labeled_array, range(1, numpatches + 1))
    sizes_sort = sorted(sizes, reverse=True)
    kmin = min(k, numpatches)
    output = np.zeros_like(image)
    for i in range(kmin):
        labeli = np.where(sizes == sizes_sort[i])[0] + 1
        output = output + np.asarray(labeled_array == labeli, np.uint8)
    return output

In [ ]:
g = "/yeping/new/AL-ACDC/data/ISIC/test/images/ISIC_0000001.npy"

In [ ]:
img_npy = reader(g)().read_image(g)
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)
img_npy = functional.normalize(img_npy,mean,std)
img_npy = np.ascontiguousarray(img_npy.transpose(2, 0, 1))[None]

*_, h, w = img_npy.shape

zoomed_img = zoom(img_npy, (1, 1, input_size / h, input_size / w), order=1,
                    mode='nearest')
zoomed_img = torch.from_numpy(zoomed_img).cuda()
output, _, _ = model(zoomed_img)


In [ ]:
imshow(zoomed_img[0].permute(1,2,0).detach().cpu())

In [ ]:
imshow(output[0].argmax(1).detach().cpu()[0])

In [ ]:
batch_pred_mask = output[0].argmax(axis=1)[0].detach().cpu().numpy()
pred_volume = zoom(batch_pred_mask, (h / input_size, w / input_size), order=0,
                    mode='nearest')

In [ ]:
imshow(pred_volume)

In [ ]:
batch_pred_mask.shape

In [ ]:
v = get_largest_k_components(batch_pred_mask)

In [ ]:
v.shape

In [ ]:

s = ndimage.generate_binary_structure(2,1)
v = ndimage.binary_opening(v,s)
v = ndimage.binary_closing(v,s)

In [ ]:
from matplotlib.pyplot import imshow

In [ ]:
imshow(v
)

In [ ]:
mask = np.load("/yeping/new/AL-ACDC/data/ISIC/test/mask/ISIC_0000001.npy")

In [ ]:
m = zoom(mask,(416/1024,416/1024),order=0)

In [ ]:
imshow(m)

In [ ]:
import os
import os.path as osp
import numpy as np
from scipy.ndimage import zoom


In [ ]:
target_dir = "/yeping/new/AL-ACDC/data/ISIC/test_size416"
source_dir = "/yeping/new/AL-ACDC/data/ISIC/test"

In [ ]:
for  name in os.listdir(osp.join(source_dir, "mask")):
    img_path = osp.join(osp.join(source_dir, "images",name))
    mask_path = osp.join(osp.join(source_dir, "mask",name))
    i, g = np.load(img_path),np.load(mask_path)
    g = zoom(g, (416 / i.shape[0], 416 / i.shape[1]), order=0,
                          mode='nearest')
    i = zoom(i, (416 / i.shape[0], 416 / i.shape[1],1), order=1,
                          mode='nearest')
    target_img_path = osp.join(osp.join(target_dir, "images",name))
    target_mask_path = osp.join(osp.join(target_dir, "mask",name))
    np.save(target_img_path, i)
    np.save(target_mask_path, g)

In [ ]:
from matplotlib.pyplot import  imshow

In [ ]:
i,g = np.load("/yeping/new/AL-ACDC/data/ISIC/test_size416/images/ISIC_0000001.npy"),np.load("/yeping/new/AL-ACDC/data/ISIC/test_size416/mask/ISIC_0000001.npy")

In [ ]:
imshow(i)

In [ ]:
imshow(g)

In [ ]:
from torch.nn.functional import  pad,interpolate
import torch
import numpy as np
from matplotlib.pyplot import imshow
from torchvision.utils import make_grid

In [ ]:
patch_size = 16
mask_ratio = 0.1
img = np.load('/home/yeep/project/py/deeplearning/AL-ACDC/data/ISIC/test/images/ISIC_0000017.npy')
device = "cuda"

In [ ]:
imshow(img)


In [ ]:
img = torch.from_numpy(np.ascontiguousarray(img.transpose(2,0,1))).to(device)

In [ ]:
img = img.permute(1,2,0)
h,w = img.shape[:2]

In [ ]:
patched_data = torch.stack([p for f in torch.chunk(img, patch_size)
                   for p in torch.chunk(f, patch_size, dim=1)])

In [ ]:
patch_num,c = patched_data.shape[0],patched_data.shape[-1]
num_patch_pixel = patch_size*patch_size
selected_idx = torch.randperm(patch_num)[:int(patch_num*mask_ratio)]
selected_patch = patched_data[selected_idx]
shuffled_patch = [p.view(num_patch_pixel,c)[torch.randperm(num_patch_pixel,device=device)].view(patch_size,patch_size,c)for p in selected_patch]
patched_data[selected_idx] = torch.stack(shuffled_patch)
shuffled_img = make_grid(patched_data.permute(0,3,1,2),nrow=h//patch_size,padding=0)

In [ ]:
imshow(shuffled_img.permute(1,2,0).cpu())

In [ ]:
img = np.load('/home/yeep/project/py/deeplearning/AL-ACDC/data/ISIC/test/images/ISIC_0000017.npy')
img = torch.from_numpy(img).permute(2,0,1)

In [ ]:
from torch.nn.functional import  pad,interpolate
import torch
import numpy as np
from matplotlib.pyplot import imshow
from torchvision.utils import make_grid
def tensor_shuffle(input):
    return input[torch.randperm(len(input),device=input.device)]

def local_shuffle(img: torch.Tensor, mask_ratio: float = 0.5, patch_size: int = 16, enable_patch_exchange: bool = False):
    '''
     img: c h w
     return: shuffled_img:torch.Tensor(c h w )
    '''
    img = img.to(device)
    img = img.permute(1, 2, 0)
    h, w = img.shape[:2]

    assert h % patch_size == 0 and w % patch_size == 0, f"img : {h,w} patch_size: {patch_size}"
    # 切割img成hxw/patch_size/patch_size个patch size x patch size的小块
    patched_data = torch.stack([p for f in torch.chunk(img, h//patch_size)
                                for p in torch.chunk(f, w//patch_size, dim=1)])
    patch_num, c = patched_data.shape[0], patched_data.shape[-1]
    num_patch_pixel = patch_size*patch_size

    selected_idx = torch.randperm(patch_num)[:int(patch_num*mask_ratio)]
    if enable_patch_exchange:
        patched_data = tensor_shuffle(patched_data)
    selected_patch = patched_data[selected_idx]
    shuffled_patch = [tensor_shuffle(p.view(num_patch_pixel, c)).view(patch_size, patch_size, c) for p in selected_patch]
    patched_data[selected_idx] = torch.stack(shuffled_patch)
    shuffled_img = make_grid(patched_data.permute(
        0, 3, 1, 2), nrow=w//patch_size, padding=0)
    return shuffled_img

In [ ]:
import SimpleITK as sitk

In [ ]:
img = sitk.GetArrayFromImage(sitk.ReadImage("/home/yeep/project/py/deeplearning/AL-ACDC/data/ACDCprecessed/test/patient084_frame01.nii.gz"))
img = torch.from_numpy(img[0][None]).to("cuda")

In [ ]:
out = local_shuffle(img,patch_size=16,mask_ratio=0.5)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
a = fig.add_subplot(1,2,1)
a.imshow(img.permute(1,2,0).cpu(),cmap="gray")
a2 = fig.add_subplot(1,2,2)
a2.imshow(out.cpu().permute(1,2,0),cmap="gray")
plt.show()

In [ ]:
img = np.load('/home/yeep/project/py/deeplearning/AL-ACDC/data/ISIC/test/images/ISIC_0000017.npy')
img = torch.from_numpy(img).to("cuda")

In [ ]:

img = img.permute(2,0,1)

In [ ]:
o = local_shuffle(img)

In [ ]:

imshow(o.permute(1,2,0).cpu())

In [ ]:
import SimpleITK as sitk

In [ ]:
from dataset.ACDCDataset import ISICDataset
import  albumentations as A

In [ ]:
train_transform = A.Compose([
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.RandomRotate90(p=0.2),
    A.ColorJitter(brightness=32 / 255, saturation=0.5),
    A.Normalize(max_pixel_value=1),
])

In [ ]:
d = ISICDataset("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/Cadispreprocessed/train",transform=train_transform)

In [ ]:
for ii,i in enumerate(d):
    img,lab = i
    if list(img.shape)!=[3, 180, 320]:
        print(ii)

In [ ]:
d[2]

In [ ]:
from sklearn.cluster import kmeans_plusplus
import torch

In [ ]:
def inter_class_var(img:torch.Tensor):
    '''img: BxCxHxW'''
    

In [ ]:
import torch
from torch.nn.functional import pairwise_distance

In [ ]:
pseudo_label = pred > 0.5

In [ ]:
def  class_var_score(pred, image):
    sample_score = []
    for p, i in zip(pred, image):
        label = p > 0.5

        class_center = []
        class_var = []
        for l in label:
            class_pix = i[:, l]
            class_center.append(class_pix.mean(1))

            var = torch.std(class_pix, dim=1).mean()
            class_var.append(var)

        # 类间散度：中心点之间的距离
        class_center = torch.stack(class_center)
        center = class_center.mean(0)
        between_class = pairwise_distance(class_center, center[None]).sum()
        # 类内方差
        inner_var = torch.mean(torch.as_tensor(class_var))
        inner_var = inner_var*0.999 + 1e-5
        score = between_class / inner_var
        sample_score.append(score)
    return sample_score

In [ ]:
pred = torch.randn(16,2,96,96).softmax(1)
image = torch.randint(0,255,(16,3,96,96),dtype=torch.float)

In [ ]:
class_var_score(pred, image)

In [ ]:
pseudo_label = pred > 0.5
foreground = (pred[:,:,None] * image[:,None])
background = (pred[:,:,None].logical_not() * image[:,None])

In [ ]:
class_center = foreground.mean(dim=[-1,-2])
center = class_center.mean(1)

In [ ]:
pred[:,torch.randn(4,96,96)>0.5].shape

In [ ]:
from pathlib import Path
from skimage.io import imread
from collections import Counter

In [ ]:
data = Path("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split2023/data")
mask_folder = list(data.rglob("mask"))

In [ ]:
z

In [ ]:
for folder in mask_folder:
    for png in folder.glob("*.png"):
        mask = imread(str(png),as_gray=True)
        counts = Counter(mask.flatten())        
        for k,v in counts.items():
            m[k] = m.get(k,0) + v
            p[k] = p.get(k,0) + 1

In [ ]:
dict(sorted(m.items(),key=lambda x : x[0]))

In [ ]:
def count_label(path_list):
    m = {}
    p = {}
    for png in path_list:
        mask = imread(str(png),as_gray=True)
        counts = Counter(mask.flatten())        
        for k,v in counts.items():
            m[k] = m.get(k,0) + v
            p[k] = p.get(k,0) + 1
    return  dict(sorted(m.items(),key=lambda x : x[0])), dict(sorted(p.items(),key=lambda x : x[0]))

In [ ]:
data = Path("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split2023/data")
mask_folder = list(data.rglob("mask"))
mask_path = [l  for p in mask_folder for l in p.iterdir()]

In [ ]:
m, p = count_label(mask_path)

In [ ]:
m, p

In [ ]:
m, p = count_label(Path("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split2023/data/test/mask").iterdir())

In [ ]:
m,p

In [ ]:
m, p = count_label(Path("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split2023/data/train/mask").iterdir())
m,p

In [ ]:
m, p = count_label(Path("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split2023/data/val/mask").iterdir())
m,p

In [ ]:
i = imread("/home/yeep/project/py/deeplearning/AL-ACDC/EXP/Cadis/semi/URPC/out_dir/Video8_frame001710.png")

In [ ]:
(i == 7).sum()

In [ ]:
import numpy as np
p = np.load("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split3407/preprocess/test/mask/Video8_frame001710.npy")

In [ ]:
(p == 7).sum()

In [ ]:
from skimage.io import imsave
from evalute.metric import metrics

In [ ]:
print(f"predicting {str(g)}")
img_npy = reader(g)().read_image(g)
mask_npy = reader(g)().read_image(gt)

img_npy = functional.normalize(img_npy, mean, std, max_pixel_value=1)
img_npy = np.ascontiguousarray(img_npy.transpose(2, 0, 1))[None]

img = torch.from_numpy(img_npy).to(device, torch.float32)
output, _, _ = model(img)
output = torch.stack(output).mean(0)

# output = model(img)
batch_pred_mask = output.argmax(axis=1)[0]
imsave(os.path.join(out_dir,
                    str(g.name)[:-4] + ".png"),
        batch_pred_mask.cpu().numpy().astype(np.uint8),
        check_contrast=False)

class_dice, class_assd = metrics(batch_pred_mask,
                                    mask_npy,
                                    class_num=8, tolerance=10)

In [ ]:
def TSNE_show(X, dims=2, save=False, perplexity=30):
    import matplotlib.pyplot as plt
    from sklearn import manifold
    import numpy as np

    tsne = manifold.TSNE(n_components=dims, init='pca', perplexity=perplexity,
                         random_state=319).fit_transform(np.array(X))
    plt.figure(figsize=(8, 8))
    plt.scatter(
            tsne[:, 0], tsne[:, 1], 10)
    plt.legend(loc='upper left')
    if save:
        plt.savefig("./TSNE_img.svg", dpi=600, format='svg')
    plt.show()

In [ ]:
a = torch.randn(64,128)

In [ ]:
TSNE_show(a)

In [ ]:
sorted([442, 458, 403, 140, 233, 706, 565, 300, 1253, 866, 290, 702, 1417, 1587, 1658, 1203, 1528, 589])

In [ ]:
sorted([290, 702, 1658, 1528, 300, 1203, 442, 140, 458, 565, 589, 403, 706, 1587, 1253, 1417, 233, 866])

In [ ]:
import numpy as np

In [ ]:
a = [list(np.random.randn(100,128)),list(np.random.randn(200,128)),list(np.random.randn(50,128))]

In [ ]:
b = np.cumsum([len(i)for i in a])

In [ ]:
c = np.concatenate(a)

In [ ]:
for i in np.split(c, b[:-1]):
    print(i.shape)

In [ ]:
from torch.nn.functional import one_hot
import torch


In [ ]:
one_hot(torch.empty(16,96,96,dtype=torch.long).random_(0,3),3).shape

In [ ]:
def dc(pred,label):
    pred, label = pred > 0.5, one_hot(label.squeeze(1), pred.shape[1]).permute(0,3,1,2).bool()
    tp = torch.sum(pred == label, dim=[-1,-2])
    fp = torch.sum(pred == torch.logical_not(label), dim=[-1,-2])
    fn = torch.sum(torch.logical_not(pred) == label, dim=[-1,-2])
            
    nominator = 2 * tp
    denominator = 2 * tp + fp + fn
    
    dc = (nominator + 1e-5) / (torch.clip(denominator + 1e-5, 1e-8))
    return dc[:,1:].mean(1)

In [ ]:
pred = torch.randn(16,3,96,96).softmax(1)
label = torch.empty(16,96,96,dtype=torch.long).random_(0,2)

In [ ]:
dc(pred,label)

In [ ]:
from util.vis import build_model,read_yml
import torch
from os.path import join as osjoin
from torch.utils.data import DataLoader
from dataset.ACDCDataset import ISICDataset
from sklearn.mixture import  GaussianMixture
import numpy as np
from sklearn.covariance import EmpiricalCovariance
from torch import nn
from sklearn import manifold

In [ ]:
class DefeultFeatureExtractor:
    def to(self, device):
        self.pool.eval()
        self.bn.eval()
        self.con1x1.eval()

        self.pool.to(device)
        self.bn.to(device)
        self.con1x1.to(device)
        return self

    def __init__(self, pool_size=12):
        self.pool = nn.Sequential(nn.AdaptiveAvgPool2d((pool_size, pool_size)))
        self.bn = nn.BatchNorm2d(512)
        self.con1x1 = nn.Conv2d(512, 256,
                                kernel_size=pool_size, bias=False)

    def __call__(self, model_output):
        _, _, features = model_output
        ret = self.con1x1(self.bn(self.pool(features[0]))).flatten(
            1, -1).cpu().numpy()
        return ret

In [ ]:
class MDistance:
    def __init__(self) -> None:
        self.ec = EmpiricalCovariance(assume_centered=True)

    def fit(self, X):
        self.ec.fit(X)

    def distance(self, x):
        return self.ec.mahalanobis(x)


In [ ]:

COLORS = ['aqua', 'coral', 'midnightblue', 'red', 'darkgreen',
          'darkred', 'maroon', 'purple', 'indigo', 'darkslategray', 'black']

In [ ]:
ckpoint = "/home/yeep/project/py/deeplearning/AL-ACDC/EXP/ISIC/MGUnet_avgalign/checkpoint/c3_best0.8860.pt"
cfg_path  = "/home/yeep/project/py/deeplearning/AL-ACDC/EXP/ISIC/MGUnet_avgalign/config.yml"

In [ ]:
cfg = read_yml(cfg_path)

In [ ]:
model = build_model(cfg)
model.load_state_dict(torch.load(ckpoint,map_location=cfg["Training"]["device"])["model_state_dict"])
model.eval()
torch.set_grad_enabled(False)

In [ ]:
data_dir = cfg["Dataset"]["data_dir"]
dataset = ISICDataset(trainfolder=osjoin(data_dir, "train"))
loader= DataLoader(dataset, batch_size=16,
                    persistent_workers=True, prefetch_factor=4,
                    num_workers=4)

In [ ]:
embeddings = []
e = DefeultFeatureExtractor(12).to(cfg["Training"]["device"])
for img,_ in loader:
    embeddings.append(e(model(img.cuda())))

In [ ]:
features = np.concatenate(embeddings) 

In [ ]:
features.shape

In [ ]:
tsne = manifold.TSNE(n_components=2, init='pca',
                             random_state=319).fit_transform(features)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(tsne[:,0],tsne[:,1],s=10)

In [ ]:
calc = MDistance()
calc.fit(features)

In [ ]:
distance = calc.distance(features)

In [ ]:
gmm = GaussianMixture(n_components=5,init_params="k-means++")

In [ ]:
pred = gmm.fit_predict(distance[:,None])

In [ ]:
gmm.predict_proba(distance[:,None]).shape

In [ ]:
for i in range(10):
    idx = np.where(pred==i)
    plt.scatter(tsne[idx,0],tsne[idx,1],s=10,c=COLORS[i])

In [ ]:
def intersect_top_q(a, b, q):
    for i in range(q, len(a)):
        inter=set(a[:i]) &set(b[:i])
        if len(inter) >=q:
            return list(inter)[:q]
    return a[:q]

In [ ]:
import torch

In [ ]:
a = torch.randn(4,8)
a

In [ ]:

a.sort(dim=1)

In [ ]:
import torch
import albumentations as A
import numpy as np
from matplotlib  import pyplot as plt
from skimage.io import imread
from torch.nn import functional as F

In [ ]:
train_transform = A.Compose([
    
        A.Compose([
            A.PadIfNeeded(384, 384),
            A.ShiftScaleRotate(rotate_limit=90),
            A.RandomCrop(194,384)
        ],
                  p=0.5),
        A.RandomGamma(gamma_limit=(0.7, 1.5)),
        A.ColorJitter(brightness=32 / 255, saturation=0.2, hue=0.1),
        A.Blur(blur_limit=3),
        A.OpticalDistortion(),
        A.VerticalFlip(),
        A.HorizontalFlip(),
 ])

In [ ]:
img = np.load("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split3407/preprocess/train/images/Video1_frame000090.npy")
mask = np.load("/home/yeep/project/py/deeplearning/AL-ACDC/data/Cadis/cadis/split3407/preprocess/train/mask/Video1_frame000090.npy")

In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
transformed = train_transform(image=img,mask=mask)

In [ ]:
plt.imshow(transformed["image"])

In [ ]:
feature = torch.randn(16,256,16,16)
mask = torch.randn(16,16,16)>0

In [ ]:
mask.shape

In [ ]:
x,y,z = torch.where(mask)

In [ ]:
feature[x,:,y,z].shape

In [ ]:
torch.sum(mask)

In [ ]:
def self_cosine_sim(f):
    norm_f = F.normalize(f, dim=1)
    return torch.matmul(norm_f, norm_f.T).pow(2).mean()
def inner_class_var_outer_class_div_feature(model_output):
    prediction, _, feature = model_output
    if len(prediction)>1:
        soft_pred = torch.stack(prediction).mean(0)    
    feature = feature[0]

    soft_pred = F.adaptive_avg_pool2d(
        soft_pred, output_size=feature.shape[2:]).softmax(1)
    numclass = soft_pred.shape[1]
    mask = soft_pred > 0.5
    score = []
    for m, f in zip(mask, feature):
        class_center = []
        inner_class_var = []
        for c in range(numclass):
            label = m[c]
            h, w = torch.where(label)
            center = f[:, h, w].mean(1)
            class_center.append(center)
            inner_class_var.append(self_cosine_sim(f[:, h, w]).item())
        inner_class = torch.mean(torch.as_tensor(inner_class_var))
        outer_class = self_cosine_sim(torch.stack(class_center))
        outer_class = outer_class*0.999+1e-5
        score.append(inner_class/outer_class)
    return torch.as_tensor(score)

In [ ]:
for img,_ in loader:
    model_output = model(img.cuda())
    score = inner_class_var_outer_class_div_feature(model_output)
    print(score)

In [ ]:
features.shape

In [ ]:
gmm = GaussianMixture(n_components=30,init_params="k-means++")
pred = gmm.fit_predict(features)

In [ ]:
import numpy as np

In [ ]:
np.random.randint(1, 26, 6)

In [ ]:
from PIL.ImageFilter import ModeFilter
import numpy as np
from PIL import Image

In [ ]:
filter = ModeFilter(size=4)

In [ ]:
input = np.random.randint(1,4,size=(16,16),dtype=np.uint8)
pred = Image.fromarray(input)

In [ ]:
np.array(input)

In [ ]:
a = pred.filter(filter)

In [ ]:
np.array(a)

In [ ]:
np.array(a)[::4,::4]

In [ ]:
np.bincount(np.array(pred)[::16,::16].flatten())

In [ ]:
np.array(a)[::16,::16]

In [ ]:
r = range(3,19)

In [ ]:
len(np.bincount(r))

In [ ]:
import numpy as np

In [ ]:
def mode_filter(image, kernel_size):
    '''
    image: h w 
    '''
    
    if len(kernel_size) == 1:
        h = w = kernel_size
    elif len(kernel_size) == 2:
        w, h = kernel_size
        print(kernel_size)
    else:
        assert "dim of kernel size is 1 or 2"
    o_h,o_w = image.shape[0]// h,image.shape[1]// w
    
    if o_h*h > image.shape[0]:
        o_h = o_h + 1
    
    if o_w*w > image.shape[1]:
        o_w = o_w + 1
        
    mode_arr = np.empty(shape=(o_h, o_w),dtype=np.uint8)
    i = 0
    for hi in range(0, image.shape[0], h):
        j = 0
        for wi in range(0, image.shape[1], w):
            wh,ww = hi + h,wi + w
            wh = wh if wh < image.shape[0] else image.shape[0]
            ww = ww if ww < image.shape[1] else image.shape[1]
            window = image[hi:wh, wi:ww]
            c = np.bincount(window.flatten())
            mode_arr[i,j] = np.argmax(c)
            j+=1
        i+=1            
    return mode_arr


In [ ]:
pred = np.random.randint(0,3,size=(256,256))

In [ ]:
np.bincount(pred[:16,:16].flatten())

In [ ]:
mode_filter(pred,(16,16))

In [ ]:
np.argmax(np.bincount([1,1,2,2]))

In [ ]:
d = {}

In [ ]:
d.setdefault(1,[]).append(1)

In [ ]:
d

In [ ]:
import numpy as np

In [ ]:
feature = np.random.randn(1800,1024,256)
center = np.random.randn(256,8)

In [ ]:
np.matmul(feature,center).shape

In [ ]:
np.linalg.norm(np.array([[1,1,1],[2,2,2]]),axis=1)

In [ ]:
# 不同group之间特征的相似度、关系

In [1]:
from os.path import join as osjoin
from os.path import exists as osexists
from torch.nn.functional import one_hot
from os.path import basename
from torch.utils.data import Dataset, DataLoader
import os
import ast
from albumentations import functional
from util import read_yml
from model.MGUnet import MGUNet
from util.reader import reader
import torch
import numpy as np
from torch import nn
from matplotlib import pyplot as plt
import time
from sklearn import manifold
import albumentations as A
from util import SubsetSampler
from pathlib import Path
from util import get_dataloader_ISIC
from dataset.ACDCDataset import ISICDataset
from util import jitfunc as f
from torch.nn import functional as F
import gc

In [2]:
def dc(pred, label):
    pred, label = pred > 0.5, one_hot(label.squeeze(
        1), pred.shape[1]).permute(0, 3, 1, 2).bool()
    tp = torch.sum(pred == label, dim=[-1, -2])
    fp = torch.sum(pred == torch.logical_not(label), dim=[-1, -2])
    fn = torch.sum(torch.logical_not(pred) == label, dim=[-1, -2])

    nominator = 2 * tp
    denominator = 2 * tp + fp + fn

    dc = (nominator + 1e-5) / (torch.clip(denominator + 1e-5, 1e-8))
    return dc[:, 1:].mean(1)


In [3]:
def _dif( a):
    r = [a[0], ]
    for i in range(1, len(a)):
        if len(a[i]) > len(a[i-1]):
            r.append(a[i] - a[i-1])
        else:
            r.append(a[i-1] - a[i])
    return r


In [4]:
def _get_query_record(filepath):
    if not osexists(filepath):
        raise FileNotFoundError

    with open(filepath) as fp:
        indices = fp.readlines()
    lab, unlab = indices[::2], indices[1::2]

    lab, unlab = [set(ast.literal_eval(l.strip())) for l in lab], \
        [set(ast.literal_eval(l.strip())) for l in unlab]
    # print(unlab)
    lab, unlab = _dif(lab), _dif(unlab)

    return lab[0], lab[1:], unlab[0], unlab[1:]

In [5]:
def build_model(cfg):
    device = cfg["Training"]["device"]
    model = MGUNet(cfg["Network"]).to(device)
    return model

In [6]:

class IDataset(Dataset):
    def __init__(self, trainfolder, transform=None) -> None:
        super().__init__()
        self.folder = trainfolder
        self.images = list((Path(trainfolder) / "images").glob("*.npy"))
        self.transforms = transform

    def __getitem__(self, index):
        data = np.load(str(self.images[index]))

        if self.transforms is not None:
            transformed = self.transforms(image=data)
            data = transformed["image"]

        return torch.tensor(np.transpose(data, [2, 0, 1]))

    def __len__(self):
        return len(self.images)

In [7]:
@torch.no_grad()
def embedding(model, loader, feature_extractor):
    model.eval()
    device = next(iter(model.parameters())).device

    features = []
    for img in loader:
        img = img.to(device, torch.float32)
        model_output = model(img)
        features += list(feature_extractor(model_output))

    return features

In [8]:
training_dir = "/home/yeep/project/py/deeplearning/AL-ACDC/EXP/MGCNet/MGUnetVar"

In [9]:
base_dir = training_dir
query_file = osjoin(base_dir, 'query_state')
init_labeled, labeled, init_unlabeled, unlabeled = _get_query_record(
    query_file)
ckdir = osjoin(base_dir, 'checkpoint')
ckpoints = sorted([osjoin(ckdir, i) for i in os.listdir(
    ckdir) if "best" in i], key=lambda x: int(basename(x).split('_')[0][1:]))
cycle_num = len(ckpoints)
cfg = read_yml(osjoin(base_dir, "config.yml"))

In [10]:
def build_loader(idxs, with_label=False):
    sampler = SubsetSampler(list(idxs))
    data_dir = cfg["Dataset"]["data_dir"]
    if not with_label:
        dataset = IDataset(trainfolder=osjoin(data_dir, "train"))
    elif with_label:
        dataset = ISICDataset(trainfolder=osjoin(data_dir, "train"))
    return DataLoader(dataset, batch_size=16,
                        sampler=sampler, persistent_workers=True, prefetch_factor=4,
                        num_workers=4)

In [11]:
def inference(loaders,feature_order=0):
    group_prediction = [[],[],[],[]]
    sim = []
    rdsim = []
    model.eval()
    for img,mask in loaders:
        img,mask = img.to(cfg["Training"]["device"]),mask.to(cfg["Training"]["device"])
        prediction,_,feature= model(img)
        
        for i,p in enumerate(prediction):
            group_prediction[i] += dc(p,mask).cpu().numpy().tolist()
                
        f = feature[feature_order]
        group_features = torch.split(f,int(f.shape[1]/4),dim=1)
        shapesim = []
        randsim = []
        for i in range(len(group_features)):
            for j in range(len(group_features)):
                f1 = group_features[i].permute(0,2,3,1)
                f2 = group_features[j].permute(0,2,3,1)
                hwsim = F.cosine_similarity(f1,f2,dim=3)
                s = hwsim.mean(dim=(-1,-2))
                tmp = []
                for _ in range(6):
                    x,y = torch.randint(0,f1.shape[1],size=(2,))
                    tmp.append(hwsim[...,x,y])
                randsim.append(torch.stack(tmp).mean(0))
                shapesim.append(s)
        sim.append(shapesim) 
        rdsim.append(randsim) 
        
    sample_sim = torch.concat([torch.stack(s).transpose(1,0) for s in sim]) 
    rdsim = torch.concat([torch.stack(s).transpose(1,0) for s in rdsim]) 
    group_prediction = torch.stack([torch.as_tensor(p) for p in group_prediction])
    return sample_sim,rdsim,group_prediction

In [12]:
torch.set_grad_enabled( False)

In [13]:
unlabeled_idx ,labeled_idx= init_unlabeled,init_labeled
model = build_model(cfg)
model.load_state_dict(torch.load(ckpoints[0], map_location=cfg["Training"]["device"])["model_state_dict"])

<All keys matched successfully>

In [14]:
cylce_unsample_sim,cycle_unrdsim,cycle_ungroup_prediction = [],[],[]
cylce_sample_sim,cycle_rdsim,cycle_group_prediction = [],[],[]

In [15]:
for c in range(cycle_num):
    unloaders = build_loader(unlabeled_idx,with_label=True)    
    loaders = build_loader(labeled_idx,with_label=True)    
    unsample_sim,unrdsim,ungroup_prediction = inference(unloaders)
    labsample_sim,labrdsim,labgroup_prediction = inference(loaders)
    
    cylce_unsample_sim.append(unsample_sim.cpu().numpy())
    cycle_unrdsim.append(unrdsim.cpu().numpy())
    cycle_ungroup_prediction.append(ungroup_prediction.cpu().numpy())
    
    cylce_sample_sim.append(labsample_sim.cpu().numpy())
    cycle_rdsim.append(labrdsim.cpu().numpy())
    cycle_group_prediction.append(labgroup_prediction.cpu().numpy())
    if c == 3:
        break
    unlabeled_idx = unlabeled_idx -  unlabeled[c]
    labeled_idx = labeled_idx | labeled[c]
    model.load_state_dict(torch.load(ckpoints[c+1], map_location=cfg["Training"]["device"])["model_state_dict"])
    
    torch.cuda.empty_cache()
    gc.collect()

In [20]:
mask = np.triu(np.ones(4),0) - np.eye(4)

In [52]:
(cylce_unsample_sim[3].mean(0).reshape(4,4)*(1-np.eye(4))).mean(1).argsort()

array([0, 2, 1, 3])

In [53]:
cycle_ungroup_prediction[3].mean(1).argsort()

array([1, 3, 2, 0])

In [38]:
print([a.mean()for a in cycle_ungroup_prediction])
print([a.mean()for a in cycle_group_prediction])

[0.91549116, 0.9446809, 0.9564784, 0.96298516]
[0.9969658, 0.9899896, 0.993434, 0.9905345]


In [ ]:
sample_mean = [i.mean(0) for i in cylce_unsample_sim]

In [ ]:
[[(sample_mean[j].reshape(4,4)*(1-np.eye(4))[i]).mean() for i in range(4)] for j in range(4)]

In [ ]:
plt.plot([(s.reshape(-1,4,4)*mask).std() for s in cylce_unsample_sim])

In [ ]:
[(s.reshape(-1,4,4)*mask).std() for s in cylce_unsample_sim]

In [ ]:
plt.plot([(s.reshape(-1,4,4)*mask).std() for s in cycle_unrdsim])

In [ ]:
[(s.reshape(-1,4,4)*mask).std() for s in cycle_unrdsim]

In [ ]:
plt.plot([(s.reshape(-1,4,4)*mask).std() for s in cylce_sample_sim])

In [ ]:
[(s.reshape(-1,4,4)*mask).mean() for s in cycle_unrdsim]

In [ ]:
plt.plot([(s.reshape(-1,4,4)*mask).std() for s in cycle_rdsim])

In [ ]:
ungroup_prediction.mean(1)

In [1]:
import numpy as np

In [11]:
a = np.random.randn(16,256,18,18).transpose(0,2,3,1)
b = (np.random.randn(16,18,18) > 0.5)

In [12]:
a[b,:].shape

(1508, 256)

In [8]:
l = []*4

In [9]:
l[1]

IndexError: list index out of range

In [2]:
import torch
from torch.nn import functional as F

In [14]:
f = torch.randn(16,32,32,256,1)
c = torch.randn(256,4)

In [15]:
F.cosine_similarity(f,c,dim=-2).shape

torch.Size([16, 32, 32, 4])

In [12]:
f.shape

torch.Size([16, 32, 32, 256, 1])

In [13]:
f[...,None].shape

torch.Size([16, 32, 32, 256, 1, 1])

In [1]:
import numpy as np

In [2]:
a = np.random.randint(0,3,size=(16,4,32,32))

In [3]:
a.shape

(16, 4, 32, 32)

In [4]:
np.unique(a)

array([0, 1, 2])

In [5]:
import torch
from torch.nn import functional as F

In [9]:
o = torch.randint(0, 4,size=(16,4,32,32),dtype=torch.float)

In [10]:
b = F.interpolate(o,size=(16,16))

In [14]:
torch.unique(b)

tensor([0., 1., 2., 3.])